# Factor Analysis

Tags: 
```
dimensionality reduction
```

## Introduction

Sometimes the data collected for analysis may have underlying, latent, or hidden factors that contribute to the patterns in the data. The factors or features of the dataset may be able to be combined in such a way that they represent a different, latent factor that was not initially considered.

Consider the following features for a fitness dataset:

| Feature |
| --- |
| Total Number of Body-Weight Squats |
| Total Number of Push-Ups |
| Total Number of Sit-Ups |
| Total Number of Pull-Ups |
| Minutes per Mile Running |
| Total Number of Lunges |
| Max Bench Press Weight |
| Max Squat Weight |

It may be possible to reduce the number of features to the latent factors that contribute to those measurements. Using the above features as an example, it's possible that features such as total number of body-weight squats, total number of lunges, and max squat weight can be represented by a Latent Factor of lower body fitness. Push-ups, max bench press, and pull-ups may likewise be better represented by a Latent Factor of upper body fitness. Sometimes these Latent Factors cannot be directly measured, but the features used to indirectly measure them are present within the features. In some ways it is like determining what a root cause for multiple features might be.

The benefits of Factor Analysis is primarily in investigating the root causes of the features as well as when attempting to find explainable ways to reduce the dimensionality of the data primarily for data visualization purposes. It would be impossible to visualize all of the above features simultaneously, but having two axes for upper body fitness and lower body fitness may be a useful way to effectively summarize the results.

## Application

The application of Factor Analysis is very similar to how Principal Components Analysis is performed. In both approaches the 

## Assumptions

## Steps

Preamble: load python libraries and the dataset

In [ ]:
# Import python packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer

In [ ]:
# Extract data and labels from the breast cancer dataset
data = load_breast_cancer().data
labels = load_breast_cancer().feature_names

# Create a DataFrame with the data and labels
df = pd.DataFrame(data, columns=labels)

# Dixplay the first 5 rows of the DataFrame
df.head()

### Step 1
**Data Preparation**

It is generally the best practice to standardize the data you are working with before performing additional transformations. This keeps the data from each feature in the same scale, preventing certain features from dominating the later transformations.

The scaled term $Z$ in this case is equal to the difference between it's original value $X$, the average value over the feature $\bar{X}$, divided by the standard deviation of the feature $\hat{\sigma}$:
$$
Z = \frac{X - \bar{X}}{\hat{\sigma}}
$$

There are other types of scaling, but this is most common.

In [ ]:
# Standardize the DataFrame by subtracting the mean and dividing by the standard deviation
for col in df.columns:
    df[col] = ( df[col] - df[col].mean() ) / df[col].std()

# Display the first 5 rows of the standardized DataFrame
df.head()

network-node-hemi,1-1-lh,1.1-1-rh,2-1-lh,2.1-1-rh,3-1-lh,3.1-1-rh,4-1-lh,4.1-1-rh,5-1-lh,5.1-1-rh,...,16.5-3-rh,16.6-4-lh,16.7-4-rh,17-1-lh,17.1-1-rh,17.2-2-lh,17.3-2-rh,17.4-3-lh,17.5-3-rh,17.6-4-lh
0,1.146477,1.724226,0.086023,0.909846,1.007335,-1.508693,1.681734,0.934843,-1.033637,-0.048148,...,0.010200,-1.207393,1.835038,-0.711909,0.030446,0.207620,1.990501,-0.281671,2.923483,-0.809335
1,1.136079,0.818788,-1.664163,-0.329550,-1.087056,-1.185072,-0.043070,-0.665321,0.563522,0.408692,...,1.085169,-1.312511,3.018299,-0.426896,1.376380,-0.421138,0.894989,-1.061768,0.600043,-0.749945
2,1.247527,1.188686,-1.298631,-0.319820,-0.724450,-0.037035,-2.324012,-3.000008,0.554291,0.955766,...,0.533812,0.154672,1.079189,0.853203,1.015228,0.034952,-0.653651,0.348756,-1.822061,0.130185
3,0.378869,0.237533,-0.878612,-0.769683,-0.285322,0.519275,-1.481487,-2.049997,0.947325,0.287926,...,1.348673,1.129490,-0.253362,1.812035,0.126205,-0.065093,0.348671,-0.423770,-0.907845,-0.095631
4,-0.051387,-1.181505,-0.351113,-0.373429,-1.738375,0.105522,-2.219029,-1.716614,0.529796,0.083014,...,1.805173,0.873977,-0.549790,1.411324,1.549633,0.753508,-0.339738,0.146762,0.124460,1.787865


There are other, built-in functions to perform scaling, the above is shown to illustrate what is actually happening. A typical workflow will use the following:

In [ ]:
from sklearn.preprocessing import StandardScaler
# Create a StandardScaler object
scaler = StandardScaler()
# Fit the scaler to the DataFrame and transform it
df_scaled = scaler.fit_transform(df)    # Output is a numpy array

# Convert the numpy array back to a DataFrame with the original labels
df_scaled = pd.DataFrame(df_scaled, columns=labels)
df_scaled.head()

### Step 2
**Factor Loadings**

Each factor $X_n$ is a linear combination of $m$ Latent Factors with an error term $\epsilon_n$ such that:
$$
X_n = \lambda_{n,1} f_1 + \dots + \lambda_{n,m} f_m + \epsilon_n
$$

Or in matrix notation:
$$
\mathbf{X} = \mathbf{\Lambda} \mathbf{f} + \mathbf{\epsilon}
$$

The $\lambda_{n,m}$ coefficients are referred to as the *factor loadings*, and the error terms $\epsilon_n$ are called the *unique factors*.

Each of the factors $f_m$ are common to each of the features. By determining the factor loading values how important each feature is for each factor, or how much the factor reflects the original features.

Consider if we choose to use a number of Latent Factors equal to the initial number of features ($n = m$) then if we attempt to minimize the error terms then the factor loadings will be $1$ for one of the Latent Factors and $0$ for all others yielding $X_n = f_m$. Since one of the goals is to be able to visualize the data it is advisable to start with 2 or 3 Latent Factors.

The matrix $\mathbf{\Lambda}$ is equal to the covariance matrix of the data $\mathbf{X}$ and the Latent Factors $\mathbf{f}$.
$$
\mathbf{\Lambda} = 
\begin{bmatrix}
\lambda_{1,1} & \lambda_{1,2} & \cdots & \lambda_{1,m} \\
\lambda_{2,1} & \lambda_{2,2} & \cdots & \lambda_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
\lambda_{n,1} & \lambda_{n,2} & \cdots & \lambda_{n,m} 
\end{bmatrix}
= \text{cov}(\mathbf{X}, \mathbf{f})
$$

Using the covariance operator on the previous notation gives:
$$
\begin{align*}
    \text{cov}(\mathbf{X}) = \mathbf{\Sigma} = & \text{cov}(\mathbf{\Lambda} \mathbf{f} + \mathbf{\epsilon}) \\
    = & \text{cov}(\mathbf{\Lambda} \mathbf{f}) + \text{cov}(\mathbf{\epsilon}) \\
    = & \mathbf{\Lambda I \Lambda}^\top + \mathbf{\Psi} \\
    = & \mathbf{\Lambda \Lambda}^\top + \mathbf{\Psi}
\end{align*}
$$

Create the covariance matrix for the data set.

In [17]:
cov_matrix = np.cov(df.T)
cov_matrix.shape

(62, 62)

*Note: we use the transpose of the data so we get a covariance of the columns, not the rows.*

### Step 3

Calculate the eigenvalue and eigenvectors for the covariance matrix.

*Note: the covariance matrix is a symmetric matrix*

In [18]:
eigenvalue, eigenvector = np.linalg.eig(cov_matrix)

The larger eigenvalues are associated with the factor loadings (eigenvectors) that have the most significantly contributing factors. We therefore sort by the magnitutde of the eigenvalues in descending order.

In [19]:
# Sort descending by eigenvalue

idx = np.argsort(-eigenvalue)
fa_eigenvalue = eigenvalue[idx]
fa_eigenvector = eigenvector[:,idx]

This step can be checked by evaluating the eigen-decomposition such that:
$$
\Sigma = \bf{V} \bf{E} \bf{V}^{T}
$$
Where $\bf{V}$ is the matrix with the eigenvectors as the columns, $\bf{E}$ is the matrix with the eigenvalues associated with each column on the diagonal, and $\Sigma$ is the covariance matrix.

In [20]:
# Create the E matrix

e_matrix = np.diag(fa_eigenvalue)

In [21]:
# Calculate the matrix multiplication

sigma_matrix = fa_eigenvector @ e_matrix @ fa_eigenvector.T

In [22]:
# Compare to the covariance matrix
# the np.allclose method is used due to floating point precision errors

np.allclose(sigma_matrix, cov_matrix)

True

### Step 4

Determine the number of factors to use.

A number of methods exist to determine how many factors to use, a handful follow:

**Kaiser's Criteria** uses all eigenvalues (and corresponding eigenvectors) that are $\ge 1$.

In [23]:
kaiser_values = [i for i in fa_eigenvalue if i >= 1]

print("Kaiser Criteria eigenvalues are:", kaiser_values)

Kaiser Criteria eigenvalues are: [10.399991640203382, 7.743590501606728, 4.325089934427531, 3.8531222876608306, 3.1320324428345905, 2.676425667354694, 2.340985108788398, 2.040920038881663, 1.6382493065525199, 1.6097022483529324, 1.3531497913378536, 1.3168355749163363, 1.227574437507627, 1.0557234760368766]


**Jolliffe's Criteria** uses all eigenvalues (and corresponding eigenvectors) that are $\ge 0.7$.

In [24]:
jolliffe_values = [i for i in fa_eigenvalue if i >= 1]

print("Jolliffe's Criteria eigenvalues are:", jolliffe_values)

Jolliffe's Criteria eigenvalues are: [10.399991640203382, 7.743590501606728, 4.325089934427531, 3.8531222876608306, 3.1320324428345905, 2.676425667354694, 2.340985108788398, 2.040920038881663, 1.6382493065525199, 1.6097022483529324, 1.3531497913378536, 1.3168355749163363, 1.227574437507627, 1.0557234760368766]


**Scree Plot** plots all of the eigenvalues to visually determine where a "bend" or "elbow" occurs before it flattens out. This is used to gauge when eigenvalues stop having a meaningful contribution to the number of factors.

In [25]:
# Create a cumulative percentage series

cumsum_percent = ( fa_eigenvalue.cumsum() / fa_eigenvalue.sum() ) * 100

In [26]:
# Create x-axis labels for the chart

scree_x = ["F"+str(i) for i in range(1,len(fa_eigenvalue)+1)]

In [27]:
sns.barplot(x="Factor", y="Eigenvalue", data=(scree_x[:50], fa_eigenvalue[:50]))

TypeError: Data source must be a DataFrame or Mapping, not <class 'tuple'>.

In [ ]:
sigma = np.array([[1,0.2,0.8],[0.2,1,0.3],[0.8,0.3,1]])
print(sigma)

In [3]:
eigenvalue, eigenvector = np.linalg.eig(sigma)

In [ ]:
print(eigenvalue)
print(eigenvector)

In [12]:
idx = np.argsort(-eigenvalue)

In [13]:
fa_eigenvalue = eigenvalue[idx]
fa_eigenvector = eigenvector[:,idx]

In [ ]:
e_matrix = np.diag(fa_eigenvalue)
print(e_matrix)

In [ ]:
v_matrix = np.array(fa_eigenvector)
print(v_matrix)

In [ ]:
v_matrix @ e_matrix @ v_matrix.T

In [18]:
e_matrix_red = e_matrix[:,0:2]

In [ ]:
v_matrix_red = v_matrix[:,0:2]

In [ ]:
loadings = fa_eigenvector[:,0:2] * np.sqrt(fa_eigenvalue[0:2])
print(loadings)

In [ ]:
common_var = loadings @ loadings.T

In [ ]:
uniqueness = np.diag(1 - common_var.diagonal())

In [ ]:
print("Covariance Matrix (Sigma):\n", sigma, "\n")
print("Eigenvalue Matrix (E):\n", e_matrix, "\n")
print("Eigenvector Matrix (V):\n", v_matrix, "\n")
print("Factor Loadings (Lambda):\n", loadings, "\n")
print("Common Variance Matrix (Lambda * Lambda.T):\n", common_var, "\n")
print("Uniqueness Matrix (psi):\n", uniqueness)

In [ ]:
sigma_est = common_var + uniqueness
print(sigma_est)

In [ ]:
print("Communalities are:", common_var.diagonal())

In [ ]:
feature1_variance = np.sum(np.square(loadings[:,0]))
feature2_variance = np.sum(np.square(loadings[:,1]))

print(feature1_variance, feature2_variance)